In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

!pip install efficientnet -q
!pip install iterative-stratification -q
!pip install tensorflow_addons -q
!pip install focal-loss
import numpy as np
import pandas
import tensorflow as tf
import os
import tensorflow_addons as tfa
import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold,MultilabelStratifiedShuffleSplit
from focal_loss import BinaryFocalLoss

In [ ]:
def onehot(image,label):
    CLASSES = 19
    return image,tf.one_hot(label,CLASSES)
def cutmix(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMSIZE[IMS]
    CLASSES = 19
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        lab1 = tf.cast(lab1,tf.float32)
        lab2 = tf.cast(lab2,tf.float32)
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2


def mixup(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = IMSIZE[IMS]
    CLASSES = 19
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        # MAKE CUTMIX LABEL
        if len(label.shape)==1:
            lab1 = tf.one_hot(label[j],CLASSES)
            lab2 = tf.one_hot(label[k],CLASSES)
        else:
            lab1 = label[j,]
            lab2 = label[k,]
        lab1 = tf.cast(lab1,tf.float32)
        lab2 = tf.cast(lab2,tf.float32)
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image2,label2


def transform(image,label):
    # THIS FUNCTION APPLIES BOTH CUTMIX AND MIXUP
    DIM = IMSIZE[IMS]
    CLASSES = 19
    SWITCH = 0.5
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
    image2, label2 = cutmix(image, label, CUTMIX_PROB)
    image3, label3 = mixup(image, label, MIXUP_PROB)
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
        labs.append(P*label2[j,]+(1-P)*label3[j,])
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,CLASSES))
    return image4,label4



In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")
        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

DIM =256
n_class = 19
def _parse_image_function(example_proto,augment = True):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string)
    }
    single_example = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.reshape( tf.io.decode_raw(single_example['image'],out_type=np.dtype('uint8')), (DIM,DIM, 3))
    mask =  tf.reshape(tf.io.decode_raw(single_example['label'],out_type=np.dtype('uint8')),[n_class])
    image = tf.dtypes.cast(image, tf.float32)
    mask = tf.dtypes.cast(mask, tf.float32)
    image = image/255.
    if augment: # https://www.kaggle.com/kool777/training-hubmap-eda-tf-keras-tpu

        if tf.random.uniform(()) > 0.5:
            image = tf.image.flip_left_right(image)
            mask = tf.image.flip_left_right(mask)

        if tf.random.uniform(()) > 0.4:
            image = tf.image.flip_up_down(image)
            mask = tf.image.flip_up_down(mask)

        if tf.random.uniform(()) > 0.5:
            image = tf.image.rot90(image, k=1)
            mask = tf.image.rot90(mask, k=1)

        if tf.random.uniform(()) > 0.45:
            image = tf.image.random_saturation(image, 0.7, 1.3)

        if tf.random.uniform(()) > 0.45:
            image = tf.image.random_contrast(image, 0.8, 1.2)
    
    return tf.cast(image, tf.float32),tf.cast(mask, tf.float32)

def load_dataset(filenames, ordered=False, augment = False):
    AUTO = tf.data.experimental.AUTOTUNE
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(lambda ex: _parse_image_function(ex, augment = augment), num_parallel_calls=AUTO)
    return dataset






def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True,augment_mixup_cutmix=False
                  , repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(True)
    
    AUTO = tf.data.experimental.AUTOTUNE
    #slices = paths if labels is None else (paths, labels)
    dset = load_dataset(paths)
    #dset = tf.data.Dataset.from_tensor_slices(slices)
    #dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    #dset = dset.map(transform, num_parallel_calls=AUTO) if augment else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize)
    dset = dset.map(transform,num_parallel_calls=AUTO) if augment_mixup_cutmix else dset
    dset = dset.prefetch(AUTO)
    
    return dset

In [ ]:
def count_data_items(filenames):
    img_num = 0
    for i in filenames:
        img_num +=int(i.split('-')[-1].split('.tf')[0])
    return img_num

In [ ]:
'''jpg
gs_list=[
'gs://kds-a5e7e2dcae43f711e19239e1403cdcb801efc84fbed5c79873c8e0e0',
'gs://kds-b79fc4bc0e8a9d183d87e6a9ed366335e32f053e3b43d89dbb444402',
'gs://kds-9df711802837f97da8bb5ba364a7d923b628992cc836779524f186bf',
'gs://kds-fb1626515f2d252abe62121eaf1b55af664a395f0480eaaa94862ac3',
'gs://kds-a6e4790428249fa3b1e531e73da79f60d1e573a79689cc200de28bb2'
]
''' 
'''
gs_list = [
'gs://kds-ee2bbb7b1b93a3aca6594b057105767bf7740a87e2c9e737df2bd14e',
'gs://kds-0127ffcbb8111d1467f791a3c1827f88fd682f420d074fcf3f5d5114',
'gs://kds-45c77f30a5cdf65369189f85ee40fd4a0f2cf13eb58e6be037942c27',
'gs://kds-90dcc93f1059ed7ab006ccc06e8013f83dc003e3c2000c6f7fa1e220',
'gs://kds-fb300e8fd046e2a7ae9baa24ab75f0e2fffdd643d56429fa3bf52edb'           
           
]
'''
#['gs://kds-a3cbdd21eaaacbeed971d397811f1d0c5629f5f3cd94676470c11403'] hpasinglecell0png
#gs://kds-37d0364b7a76e93071da034c72923c99e801a09c94f2aef2e6e257ea hpasinglecell1png
#'gs://kds-19fc2671d87d00f0318a60fe2a3b9dea6ad96536a1faa53532417be9' hpasinglecell2png
#gs://kds-783a2216109b925edb061147dd4bd7a5fd9fd22499918bc30af3caa7 hpasinglecell3png
#'gs://kds-7ab9ae4e26848338cea01aa1d8ceef869749ed8aa28b8b7452e3ef2f' hpasinglecell4png

In [ ]:
# glob the data
'''
x_list=[]
for i in gs_list:
  x_list.extend(tf.io.gfile.glob(i+'/cell_all_128/*'))
  '''

In [ ]:
'''
df = pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')
df = df[['ID','Label']]
for i in range(19):
    df[f'{i}'] =0

for i in tqdm(range(df.shape[0])):
    #df.loc[i,'ID']=df.loc[i,'Image'].split('/')[-1]+'_green'
    a = str(df.loc[i,'Label']).split('|')
    a_len = len(str(df.loc[i,'Label']).split('|'))
    for j in range(a_len ):
        df.loc[i,a[j]] = 1

df_onehot = df.copy()[['ID','Label','0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18']]
'''

In [ ]:
'''
label_cols = df_onehot.columns[2:21]
y_list=[]
for i in x_list:
  ID = i.split('.png_')[0].split('/')[-1].split('_')[-2]
  y_list.append(df_onehot[df_onehot['ID']==ID][label_cols].values)

#y_list = np.load('y_list.npz')['arr_0']

#y_list = y_list.astype(np.float32)
'''

In [ ]:
#y_list = np.array(y_list)
#y_list = y_list.astype(np.float32)

In [ ]:
#y_list = np.squeeze(y_list)

In [ ]:
#x_list = np.array(x_list)

In [ ]:
'''
(
    train_paths, valid_paths, 
    train_labels, valid_labels
) = train_test_split(x_list, y_list, test_size=0.2, random_state=42)
'''

In [ ]:
#kf = MultilabelStratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=100)

In [ ]:
'''
for train_index, test_index in kf.split(x_list, y_list):
   
   train_paths, valid_paths = x_list[train_index], x_list[test_index]
   train_labels, valid_labels = y_list[train_index], y_list[test_index]
   break
'''

In [ ]:
#'gs://kds-d2130e02868047c380c97c6ae5d31a4180f74d21ceefbf5d4c9e531e' 0
#'gs://kds-53777409f1b5814b19e2bc8b542d4af1163b9ac03623463085b45210' ../input/hpatrain-tfrec-1
#'gs://kds-37f6a0e35a9bb77b47960da902f2fad44df121760c080ff2a9a04257' hpatrain-tfrec-2
#'gs://kds-e3c610c144b38135540fa8471f82968eab5f181530e876ba09d20fad' hpatrain-tfrec-3

train_list=[]
#ori
train_list.extend(tf.io.gfile.glob('gs://kds-18467a34620ba1afe1f389fd285bf108447b09f584cbe7c55a500ebe/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-099b48978fad967933337af13a66cb036007b1310d5b231711934bb2/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-695d786331c70c00dde5644812f3b8ea8f2ae892cab2a4f48a2f9766/MYS*'))


#ex
train_list.extend(tf.io.gfile.glob('gs://kds-7b0babb233b7ca137d3618494467528c94939ab3ed6700474cc134e3/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-1f4282d99006b8514585b07f4a361bcfb8d19454bb7872b81e7d37e7/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-ac7d763f7f8eb506cb0b1868e31b45301e079bb1d509df6237aaf8db/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-f2ed388129c9136f54e2645a1d8b30622c264e9dfcf1d59daab50c4f/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-52c5b56223349671ebe5f640fb11981cd40a1c052da071daeca2ba8a/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-81e6dfe132eb32b60d9d018d968e64a7633e530b5a212a50d6e9ded9/MYS*'))

train_list.extend(tf.io.gfile.glob('gs://kds-f2ed388129c9136f54e2645a1d8b30622c264e9dfcf1d59daab50c4f/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-52c5b56223349671ebe5f640fb11981cd40a1c052da071daeca2ba8a/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-81e6dfe132eb32b60d9d018d968e64a7633e530b5a212a50d6e9ded9/MYS*'))
#train_list = tf.io.gfile.glob('gs://kds-6922a5ae3ea86268c632cb56f65631de76976e8d47d9b775ff699612/train_*')

#ex
valid_list = []
train_list.extend(tf.io.gfile.glob('gs://kds-da16be69b2e2a909bc95a2cffd9efede5d035fcb9793591520d96332/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-a28dfde12325e031ae832c7e818f84c660a297284d409b20ed496c63/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-dd992e3678f2e10af911cd0f85c5925d9bf9ed530161c790b780c23f/MYS*'))
valid_list.extend(tf.io.gfile.glob('gs://kds-df3fc43230853e82f685f4c3029c6238b8d29cfb98d20ae635a0d2d2/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-30b9f186e0fe2978ad22c7f59109e4cc4e65ec9896839e5ca0b10ccb/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-06683f7151402f921a76bdb3dbee0b711d13e8de449343e09a8404dd/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-60c69c7f8822b869b35a0c1390c61041abf3b2932c1caced16df832d/MYS*'))
train_list.extend(tf.io.gfile.glob('gs://kds-c9807109d89d8da8c8077075206ea4c048cd05fce211953e601a5f75/MYS*'))


In [ ]:
'''
'gs://kds-18467a34620ba1afe1f389fd285bf108447b09f584cbe7c55a500ebe' MYS0101
'gs://kds-099b48978fad967933337af13a66cb036007b1310d5b231711934bb2' MYS0102
'gs://kds-695d786331c70c00dde5644812f3b8ea8f2ae892cab2a4f48a2f9766' MYS0103
'gs://kds-7b0babb233b7ca137d3618494467528c94939ab3ed6700474cc134e3' MYS0104
'gs://kds-1f4282d99006b8514585b07f4a361bcfb8d19454bb7872b81e7d37e7' MYS0105
'gs://kds-ac7d763f7f8eb506cb0b1868e31b45301e079bb1d509df6237aaf8db' MYS0106
'gs://kds-f2ed388129c9136f54e2645a1d8b30622c264e9dfcf1d59daab50c4f' MYS0107
'gs://kds-52c5b56223349671ebe5f640fb11981cd40a1c052da071daeca2ba8a' MYS0108

'gs://kds-81e6dfe132eb32b60d9d018d968e64a7633e530b5a212a50d6e9ded9' MYS0201
'gs://kds-da16be69b2e2a909bc95a2cffd9efede5d035fcb9793591520d96332' MYS0202
'gs://kds-a28dfde12325e031ae832c7e818f84c660a297284d409b20ed496c63' MYS0203
'gs://kds-dd992e3678f2e10af911cd0f85c5925d9bf9ed530161c790b780c23f' MYS0204
'gs://kds-df3fc43230853e82f685f4c3029c6238b8d29cfb98d20ae635a0d2d2' MYS0205
'gs://kds-30b9f186e0fe2978ad22c7f59109e4cc4e65ec9896839e5ca0b10ccb' MYS0206
'gs://kds-06683f7151402f921a76bdb3dbee0b711d13e8de449343e09a8404dd' MYS0207
'gs://kds-60c69c7f8822b869b35a0c1390c61041abf3b2932c1caced16df832d' MYS0208
'gs://kds-c9807109d89d8da8c8077075206ea4c048cd05fce211953e601a5f75' MYS0209
'''

In [ ]:
#'gs://kds-c90356b0c1a5565426b91f2ffea10276766a0b17367e43f88882d164' hpaexset-02tfrec
#'gs://kds-42aa49ce9fc8212676e97da0be4f4b4ddbcb0f04aa5a7481a97aac08' hpaexset-3-5tfrec
#'gs://kds-bc31e123b5089b0128a4f3e872c3dfbb6b3545a2e87468449cbda814' hpaexset-6-8tfrec
#'gs://kds-871d044987171af8883d7c7f98174dc75853a4a5eb5f28f6a30d0eb7' hpaexset-9-11
#'gs://kds-75bbca92fb91ce73f2e1c5eaa27578caa91308d0f702d9aa0b77fbce' hpaexset-12-14tfrec
#'gs://kds-cd5df74554678136acf099d07fdc0b632dd162396e04f51616040fa8' hpaexset-15-17tfrec
#'gs://kds-96c49accad963db1c4068dadc7132d6495323b4785eae6522b537115' hpaexset-18-20tfrec
#'gs://kds-c0fa84156d8e93032e87c17985a0c4051c300863de62a3f341d94acc' hpaexset-21-23tfrec
# 'gs://kds-a6c822533c6fa25a6358bb47e20d988305def64a9790104bfb4e3596' hpaexset-24-26tfrec
#'gs://kds-de5b3c0fdfbe51f9fc2e36af8665b2b76bf50daed0f7b2b705c8e0c6' hpaexset-27-30tfrec
# 'gs://kds-5566487c260e590cfab3f0b9c79fc433988134436a6ee939860b5d44' hpaexset-31-34tfrec
#'gs://kds-bc6e8d8aeb83e22f6839287353c7ee17784d3c1d852528a12679a6a8' hpaexset-35-37tfrec

In [ ]:
IMSIZE = (224, 240, 260, 300, 380, 456, 528, 600, 256)
IMS = 8

strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 32
AUG_BATCH = BATCH_SIZE




train_dataset = build_dataset(
    train_list,  bsize=BATCH_SIZE,cache=False,augment_mixup_cutmix=True
)

valid_dataset = build_dataset(
    valid_list,  bsize=BATCH_SIZE,
    cache=False,repeat=False, shuffle=False, augment=False
)

#decoder = build_decoder(with_labels=True, target_size=(IMSIZE[IMS], IMSIZE[IMS]))
#test_decoder = build_decoder(with_labels=False, target_size=(IMSIZE[IMS], IMSIZE[IMS]))
'''
train_dataset = build_dataset(
    train_paths, train_labels, bsize=BATCH_SIZE, decode_fn=decoder,cache=False,augment_mixup_cutmix=True
)
'''
''' dataset checking
from tqdm import tqdm
row = 6; col = 4;
row = min(row,BATCH_SIZE//col)
for (img,label) in tqdm(train_dataset):
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    break
'''
'''
valid_dataset = build_dataset(
    valid_paths, valid_labels, bsize=BATCH_SIZE, decode_fn=decoder,
    cache=False,repeat=False, shuffle=False, augment=False
)
'''

In [ ]:
'''
with strategy.scope():
    
    model = tf.keras.models.load_model(
            '../input/hpa-training-tpu/model_RGB_0.2186.h5'
        )
'''

In [ ]:
from tqdm import tqdm

In [ ]:
'''
high_confidence=0
img_num=[]
label=[]
total= len(result)*19
for i in tqdm(range(len(result))):
    first=1
    for j in range(len(result[i])):
        if result[i][j]>0.8:
            
            if first:
                img_num.append(i)
                label.append([])
                first=0
            label[-1].append(j)
            high_confidence+=1
'''

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def en_example(l,img):
    feature = {
      'label': _bytes_feature(l.tobytes()),
      'image': _bytes_feature(img.tobytes()),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
'''
kt=[]
record_file='temp.tfrecords'
cnt = 0
with tf.io.TFRecordWriter(record_file) as writer:
    for i in valid_dataset:
    
        kt.append(model.predict(i[0]))

        for value,pred,true in zip(i[0],kt[-1],i[1]):

                true_amount_b = np.sum(true)
                true += pred>0.8
                true -= pred<0.1
                true = np.array(true)
                true[true<0] = 0.
                true[true>1] = 1.
                if np.sum(true) ==0:
                    if true[18]>0.2:
                        true[18]=1
                true_amount_a = np.sum(true)
                print(true_amount_a)
                if true_amount_a == 0:
                    continue
                
                true = np.array(true,dtype='uint8')
                value = np.array(value,dtype='uint8')

                exp = en_example(true,value)
                writer.write(exp.SerializeToString())
                cnt+=1
                print(cnt)
            
        
        break
os.rename(record_file,'-'+str(cnt)+'.tfrecords')
'''

In [ ]:
# define the model

try:
    n_labels = 19#train_labels.shape[1]
except:
    n_labels = 1
    
with strategy.scope():
    '''
    model = tf.keras.models.load_model(
            '/content/gdrive/MyDrive/Kaggle/model_RGB_0.0051.h5'
        )
    '''
    
    model = tf.keras.Sequential([
        efn.EfficientNetB0(
            input_shape=(IMSIZE[IMS], IMSIZE[IMS], 3),
            weights='imagenet',
            include_top=False),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(n_labels, activation='sigmoid')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',#BinaryFocalLoss(gamma=5),#'binary_crossentropy',#tfa.losses.SigmoidFocalCrossEntropy(),
        metrics=[tf.keras.metrics.AUC(multi_label=True)])
        
    model.summary()

In [ ]:
colour = '_RGB'
#steps_per_epoch = len(train_paths) // BATCH_SIZE
steps_per_epoch = count_data_items(train_list) // BATCH_SIZE
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "model_RGB_{val_loss:.4f}.h5", save_best_only=False, monitor='val_loss', mode='min')
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=3, min_lr=1e-6, mode='min')

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=2,
    verbose=1,
    callbacks=[checkpoint, lr_reducer],
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_dataset)

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.to_csv(f'history{colour}.csv')